<a href="https://colab.research.google.com/drive/1II7OeTtyQXcNkYDe0WNQPBz1raLx0ejU?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### Tree of Thoughts (ToT)

In [1]:
!pip install -qU google-generativeai

In [2]:
import google.generativeai as genai
import getpass

Get free-tier Google's Gemini API Key here: https://aistudio.google.com/app/apikey

In [3]:
API_KEY = getpass.getpass("Enter your Google API key: ")

Enter your Google API key: ··········


In [4]:
genai.configure(api_key=API_KEY)

In [5]:
class ThoughtNode:
    def __init__(self, content, depth=0, parent=None):
        self.content = content
        self.depth = depth
        self.parent = parent
        self.children = []
        self.value = 0.0  # Evaluation score

    def add_child(self, child):
        self.children.append(child)
        child.parent = self

class ToTAgent:
    def __init__(self):
        self.model = genai.GenerativeModel("gemini-2.0-flash-exp")
        self.root = None

    def generate_thoughts(self, problem, current_thought, num_thoughts=3):
        """Generate multiple candidate thoughts"""
        prompt = f"""Problem: {problem}

        Current reasoning: {current_thought}

        Generate {num_thoughts} different next reasoning steps or ideas.
        List them numbered 1-{num_thoughts}:"""

        response = self.model.generate_content(prompt).text

        # Parse thoughts
        thoughts = []
        for line in response.split("\n"):
            line = line.strip()
            if line and (line[0].isdigit() or line.startswith("-")):
                # Remove numbering
                thought = line.lstrip("0123456789.-) ").strip()
                if thought:
                    thoughts.append(thought)

        return thoughts[:num_thoughts]

    def evaluate_thought(self, problem, thought):
        """Evaluate quality of a thought (0-10)"""
        prompt = f"""Problem: {problem}

        Reasoning step: {thought}

        Rate this reasoning step's quality (0-10):
        - Does it make progress toward solving the problem?
        - Is it logical and valid?
        - Does it seem promising?

        Score (just the number):"""

        response = self.model.generate_content(prompt).text

        try:
            score = float(response.strip().split()[0])
            return min(max(score / 10, 0), 1)  # Normalize to 0-1
        except:
            return 0.5

    def bfs_search(self, problem, max_depth=3, branch_factor=3):
        """Breadth-First Search through thought tree"""
        print(f"\n{'='*60}")
        print(f"🌳 Tree of Thoughts (BFS)")
        print(f"{'='*60}")
        print(f"Problem: {problem}\n")

        # Initialize root
        self.root = ThoughtNode("Starting to solve the problem", depth=0)
        queue = [self.root]

        best_path = []
        best_score = 0

        while queue and queue[0].depth < max_depth:
            node = queue.pop(0)

            print(f"{'  ' * node.depth}📍 Depth {node.depth}: {node.content[:60]}...")

            # Generate candidate thoughts
            thoughts = self.generate_thoughts(problem, node.content, branch_factor)

            for thought in thoughts:
                # Create child node
                child = ThoughtNode(thought, node.depth + 1, node)
                node.add_child(child)

                # Evaluate thought
                score = self.evaluate_thought(problem, thought)
                child.value = score

                print(f"{'  ' * child.depth}  ├─ [Score: {score:.2f}] {thought[:50]}...")

                # Track best path
                path_score = self._path_score(child)
                if path_score > best_score:
                    best_score = path_score
                    best_path = self._get_path(child)

                # Add to queue if promising
                if score > 0.4:
                    queue.append(child)

            print()

        return best_path, best_score

    def dfs_search(self, problem, max_depth=3, branch_factor=3):
        """Depth-First Search with backtracking"""
        print(f"\n{'='*60}")
        print(f"🌲 Tree of Thoughts (DFS)")
        print(f"{'='*60}")
        print(f"Problem: {problem}\n")

        self.root = ThoughtNode("Starting to solve the problem", depth=0)

        best_path = []
        best_score = 0

        def dfs(node):
            nonlocal best_path, best_score

            if node.depth >= max_depth:
                # Reached max depth
                path_score = self._path_score(node)
                if path_score > best_score:
                    best_score = path_score
                    best_path = self._get_path(node)
                return

            print(f"{'  ' * node.depth}📍 Depth {node.depth}: {node.content[:60]}...")

            # Generate and evaluate thoughts
            thoughts = self.generate_thoughts(problem, node.content, branch_factor)

            evaluated_thoughts = []
            for thought in thoughts:
                score = self.evaluate_thought(problem, thought)
                evaluated_thoughts.append((thought, score))
                print(f"{'  ' * (node.depth + 1)}  ├─ [Score: {score:.2f}] {thought[:50]}...")

            # Sort by score (best first)
            evaluated_thoughts.sort(key=lambda x: x[1], reverse=True)

            print()

            # Explore best thoughts (backtrack from poor ones)
            for thought, score in evaluated_thoughts:
                if score > 0.3:  # Threshold for exploration
                    child = ThoughtNode(thought, node.depth + 1, node)
                    child.value = score
                    node.add_child(child)

                    # Recursively explore
                    dfs(child)
                else:
                    print(f"{'  ' * (node.depth + 1)}  ⚠️  Backtracking from low-value path\n")

        dfs(self.root)
        return best_path, best_score

    def _path_score(self, node):
        """Calculate cumulative score along path"""
        score = 0
        count = 0
        while node:
            score += node.value
            count += 1
            node = node.parent
        return score / count if count > 0 else 0

    def _get_path(self, node):
        """Get path from root to node"""
        path = []
        while node:
            path.append(node.content)
            node = node.parent
        return list(reversed(path))

    def solve(self, problem, method="bfs", max_depth=3):
        """Solve problem using ToT"""
        if method == "bfs":
            path, score = self.bfs_search(problem, max_depth)
        else:
            path, score = self.dfs_search(problem, max_depth)

        print(f"{'='*60}")
        print(f"🏆 BEST REASONING PATH (Score: {score:.2f})")
        print(f"{'='*60}")
        for i, step in enumerate(path):
            print(f"{i}. {step}")
        print()

        # Generate final answer
        path_text = "\n".join([f"{i+1}. {step}" for i, step in enumerate(path)])

        final_prompt = f"""Problem: {problem}

        Reasoning path:
        {path_text}

        Based on this reasoning, provide the final answer:"""

        final_answer = self.model.generate_content(final_prompt).text

        print(f"{'='*60}")
        print(f"💡 FINAL ANSWER")
        print(f"{'='*60}")
        print(final_answer)
        print()

        return final_answer

In [6]:
# Example 1: Math Puzzle (24 Game)
print("="*60)
print("EXAMPLE 1: Math Puzzle - Make 24")
print("="*60)

tot1 = ToTAgent()
tot1.solve(
    "Use the numbers 4, 6, 8, 8 with operations +, -, *, / to make 24. Each number must be used exactly once.",
    method="bfs",
    max_depth=2
)


# Example 2: Logic Puzzle
print("\n" + "="*60)
print("EXAMPLE 2: Logic Puzzle")
print("="*60)

tot2 = ToTAgent()
tot2.solve(
    "Three people: Alice, Bob, Carol. One always tells truth, one always lies, one alternates. "
    "Alice says 'Bob is the liar'. Bob says 'Carol alternates'. Who is who?",
    method="dfs",
    max_depth=2
)


# Example 3: Creative Writing
print("\n" + "="*60)
print("EXAMPLE 3: Creative Story Planning")
print("="*60)

tot3 = ToTAgent()
tot3.solve(
    "Write an opening scene for a sci-fi story. The protagonist discovers something mysterious. "
    "Explore different narrative approaches.",
    method="bfs",
    max_depth=2
)


# Example 4: Strategic Planning
print("\n" + "="*60)
print("EXAMPLE 4: Strategic Planning")
print("="*60)

tot4 = ToTAgent()
tot4.solve(
    "A startup has $100k budget, 3 months, and 2 developers. Should they focus on: "
    "A) Mobile app, B) Web platform, or C) API service? Consider market, resources, timeline.",
    method="dfs",
    max_depth=2
)


# Example 5: Problem Decomposition
print("\n" + "="*60)
print("EXAMPLE 5: Complex Problem Decomposition")
print("="*60)

tot5 = ToTAgent()
tot5.solve(
    "How can a city reduce traffic congestion by 30% in 2 years? "
    "Explore different approaches systematically.",
    method="bfs",
    max_depth=2
)


# Example 6: Game Strategy
print("\n" + "="*60)
print("EXAMPLE 6: Chess Opening Strategy")
print("="*60)

tot6 = ToTAgent()
tot6.solve(
    "As white in chess, what's the best opening strategy against the Sicilian Defense? "
    "Evaluate different approaches.",
    method="dfs",
    max_depth=2
)


print("✅ Tree of Thoughts Complete!")

EXAMPLE 1: Math Puzzle - Make 24

🌳 Tree of Thoughts (BFS)
Problem: Use the numbers 4, 6, 8, 8 with operations +, -, *, / to make 24. Each number must be used exactly once.

📍 Depth 0: Starting to solve the problem...
    ├─ [Score: 1.00] **Focus on getting close to 24:** Try to combine t...
    ├─ [Score: 0.70] **Consider division:** Division can drastically ch...
    ├─ [Score: 0.80] **Explore multiplication and addition/subtraction:...

  📍 Depth 1: **Focus on getting close to 24:** Try to combine two numbers...
      ├─ [Score: 0.80] **Focus on using division to simplify:** Can we di...
      ├─ [Score: 0.80] **Target a number that's a factor of 24:** Explore...
      ├─ [Score: 0.80] **Consider using subtraction to get closer:** Can ...

  📍 Depth 1: **Consider division:** Division can drastically change the n...
      ├─ [Score: 0.80] **Explore the (8/8) = 1 path further:** Since we'v...
      ├─ [Score: 0.90] **Consider using multiplication to get close to 24...
      ├─ [Score:

TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 50
Please retry in 33.825910095s.